# Learning to Rank for Job–Resume Matching

This notebook implements the **end-to-end experimental pipeline** described in the thesis section *Experimental Setup* for a Learning to Rank (LTR) system that recommends suitable candidates (resumes) for each job posting.

Mục tiêu của notebook là cung cấp một pipeline **tái lập được** (reproducible) cho các bước:
- Xây dựng và mô tả tập dữ liệu job postings thực tế và tập hồ sơ ứng viên synthetic.
- Tiền xử lý và chuẩn hóa các trường dữ liệu chính (kỹ năng, kinh nghiệm, văn bản mô tả, vị trí địa lý).
- Sinh tập cặp job–resume tiềm năng bằng chiến lược hai bước (lọc theo kỹ năng và vị trí, sau đó xếp hạng theo độ tương đồng ngữ nghĩa SBERT).
- Thiết kế bộ đặc trưng cho bài toán Learning to Rank, bao phủ các khía cạnh skill matching, experience alignment, location matching và semantic similarity.
- Xây dựng nhãn relevance bậc thang (0–3) dựa trên các ngưỡng định lượng, đảm bảo phân bố nhãn hợp lý cho huấn luyện.
- Tổ chức dữ liệu theo chuẩn query–document (qid) cho các thuật toán LTR.
- Huấn luyện mô hình LambdaMART và baseline rule-based, đánh giá theo NDCG@10, MAP@10, Precision@10 và kiểm định ý nghĩa thống kê (Wilcoxon signed-rank test).

Toàn bộ cell được thiết kế theo thứ tự tuyến tính để có thể chạy từ trên xuống dưới trong môi trường Python 3.10 với các thư viện được nêu rõ, đảm bảo phù hợp để đính kèm như phần bổ sung kỹ thuật cho luận văn hoặc bài báo khoa học.

In [ ]:
# 0. Setup: imports, configuration, and paths

import os
import re
import random
from typing import List

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
from scipy.stats import wilcoxon

import lightgbm as lgb

# Optional: sentence-transformers for SBERT embeddings
try:
    from sentence_transformers import SentenceTransformer
except ImportError:
    SentenceTransformer = None
    print("sentence-transformers is not installed. Install it with `pip install sentence-transformers` if you want to run embedding steps.")

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Paths to data (adjust if your folder structure is different)
DATA_DIR = "../data"
JOB_PATH = os.path.join(DATA_DIR, "jobs_vietnamworks_formatted_fixed.csv")
RESUME_PATH = os.path.join(DATA_DIR, "synthetic_resumes.csv")

print("Job dataset:", JOB_PATH)
print("Resume dataset:", RESUME_PATH)


In [ ]:
# 1. Load and inspect datasets

jobs = pd.read_csv(JOB_PATH)
resumes = pd.read_csv(RESUME_PATH)

print("Jobs shape:", jobs.shape)
print("Resumes shape:", resumes.shape)

display(jobs.head())
display(resumes.head())

jobs.info()
resumes.info()


## 2. Data preprocessing

This section implements the *Cleaning and Normalization* and basic preparation for job–resume pairing:

- Chuẩn hóa trường kỹ năng (`skills_list`, `Skills`, `Additional Skills`, `Certifications`, `Languages`)
- Chuẩn hóa kinh nghiệm về số năm (`experience_years_min`, `experience_years_max`, `Years of Experience`)
- Làm sạch các trường text dài (`description`, `Work Experience`, `Desired Job`)
- Chuẩn hóa location về dạng tỉnh/thành để dùng cho matching theo vùng

Các hàm dưới đây được thiết kế sao cho có thể tái sử dụng trong pipeline thực tế.


In [ ]:
# 2.1 Helper functions for cleaning and normalization

VIETNAM_PROVINCES = [
    "Hồ Chí Minh", "Hà Nội", "Đà Nẵng", "Bình Dương", "Đồng Nai", "Bắc Ninh", "Bắc Giang",
    "Hưng Yên", "Kiên Giang", "Hải Phòng", "Quảng Ninh", "Bến Tre", "Nam Định"
]

SKILL_SYNONYMS = {
    # Simple handcrafted ontology, có thể mở rộng thêm cho các nhóm kỹ năng chuyên ngành khác khi cần thiết
    "giao tiếp": ["giao tiếp", "communication", "communication skills"],
    "quản lý thời gian": ["quản lý thời gian", "time management"],
    "bán hàng": ["bán hàng", "sales"],
}


def clean_text(text: str) -> str:
    if pd.isna(text):
        return ""
    text = str(text)
    text = re.sub(r"\s+", " ", text)  # collapse whitespace
    text = re.sub(r"[\r\n\t]", " ", text)
    return text.strip()


def normalize_skills(raw: str) -> List[str]:
    if pd.isna(raw):
        return []
    # Tách theo dấu phẩy hoặc dấu "|"
    parts = re.split(r"[,|]", str(raw))
    skills = []
    for p in parts:
        s = p.strip().lower()
        if not s:
            continue
        skills.append(s)
    # Ánh xạ synnonym về canonical key
    normalized = []
    for s in skills:
        mapped = False
        for canon, group in SKILL_SYNONYMS.items():
            if s in [g.lower() for g in group]:
                normalized.append(canon)
                mapped = True
                break
        if not mapped:
            normalized.append(s)
    # Loại trùng lặp, giữ thứ tự
    seen = set()
    deduped = []
    for s in normalized:
        if s not in seen:
            seen.add(s)
            deduped.append(s)
    return deduped


def normalize_location(raw: str) -> str:
    if pd.isna(raw):
        return ""
    text = str(raw)
    # Một số trường có dạng "Hồ Chí Minh | Hà Nội | Đà Nẵng"
    parts = re.split(r"[,|]", text)
    parts = [p.strip() for p in parts if p.strip()]
    for p in parts:
        for prov in VIETNAM_PROVINCES:
            if prov.lower() in p.lower():
                return prov
    # Fallback: trả về phần đầu tiên đã clean
    return parts[0] if parts else text.strip()


def experience_years_from_range(min_years: float, max_years: float) -> float:
    if pd.isna(min_years) and pd.isna(max_years):
        return np.nan
    if pd.isna(min_years):
        return float(max_years)
    if pd.isna(max_years):
        return float(min_years)
    return float((min_years + max_years) / 2.0)


# Apply cleaning to jobs and resumes
jobs["description_clean"] = jobs["description"].apply(clean_text)
jobs["location_norm"] = jobs["location_list"].apply(normalize_location)
jobs["skills_norm"] = jobs["skills_list"].apply(normalize_skills)

jobs["experience_years"] = jobs.apply(
    lambda r: experience_years_from_range(r.get("experience_years_min"), r.get("experience_years_max")),
    axis=1,
)

resumes["work_experience_clean"] = resumes["Work Experience"].apply(clean_text)
resumes["desired_job_clean"] = resumes["Desired Job"].apply(clean_text)
resumes["city_norm"] = resumes["City"].apply(normalize_location)
resumes["skills_norm"] = resumes["Skills"].apply(normalize_skills)

print("Preprocessing done.")


## 3. SBERT embeddings for jobs and resumes

Ở phần này, chúng ta sinh embedding ngữ nghĩa cho job và resume để sử dụng cho:
- Tìm kiếm gần đúng (top-N resume tiềm năng cho mỗi job)
- Các đặc trưng semantic trong mô hình Learning to Rank.

Nếu không cài `sentence-transformers`, bạn có thể bỏ qua cell này hoặc cài thêm thư viện để chạy đầy đủ pipeline.


In [ ]:
# 3.1 Build SBERT embeddings (optional but recommended)

EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

if SentenceTransformer is not None:
    sbert_model = SentenceTransformer(EMBEDDING_MODEL_NAME)

    job_texts = (
        jobs["title"].fillna("")
        + " "
        + jobs["description_clean"].fillna("")
    ).tolist()

    resume_texts = (
        resumes["Desired Job"].fillna("")
        + " "
        + resumes["work_experience_clean"].fillna("")
    ).tolist()

    job_embs = sbert_model.encode(job_texts, batch_size=64, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
    resume_embs = sbert_model.encode(resume_texts, batch_size=64, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)

    print("Job embeddings:", job_embs.shape)
    print("Resume embeddings:", resume_embs.shape)
else:
    job_embs = None
    resume_embs = None
    print("SBERT not available; semantic similarity features will be limited.")


## 4. Job–resume candidate generation

Theo Experimental Setup, mỗi job chỉ ghép với **top-N = 200** resume tiềm năng để tránh bùng nổ tổ hợp:
- Lọc sơ cấp theo **chồng lặp kỹ năng** và **vị trí địa lý**
- Sau đó chọn top-N theo **cosine similarity** từ embedding SBERT (nếu có) hoặc fallback sang điểm kỹ năng.

Để notebook vẫn chạy được trên máy hạn chế tài nguyên, có thể chọn subset job để demo (ví dụ 500 job đầu tiên).


In [ ]:
# 4.1 Candidate generation utility

from tqdm.auto import tqdm


def skill_overlap_ratio(job_skills: List[str], res_skills: List[str]) -> float:
    if not job_skills or not res_skills:
        return 0.0
    s_job = set(job_skills)
    s_res = set(res_skills)
    inter = len(s_job & s_res)
    if len(s_job) == 0:
        return 0.0
    return inter / len(s_job)


def build_candidate_pairs(
    jobs_df: pd.DataFrame,
    resumes_df: pd.DataFrame,
    job_embs: np.ndarray | None,
    resume_embs: np.ndarray | None,
    top_n: int = 200,
    min_skill_overlap: float = 0.3,
    max_jobs: int | None = None,
) -> pd.DataFrame:
    """Build job–resume candidate pairs as in Section 4.2.

    Note: to keep runtime reasonable, you can set `max_jobs` to e.g. 500.
    """
    if max_jobs is not None:
        jobs_df = jobs_df.iloc[:max_jobs].copy()

    pairs = []

    for j_idx, j_row in tqdm(jobs_df.iterrows(), total=len(jobs_df)):
        j_loc = j_row["location_norm"]
        j_skills = j_row["skills_norm"] or []

        # Filter resumes by same province (or any if missing)
        if j_loc:
            subset = resumes_df[resumes_df["city_norm"] == j_loc]
        else:
            subset = resumes_df

        if subset.empty:
            continue

        # Compute skill overlap and keep those above threshold
        overlaps = []
        for r_idx, r_row in subset.iterrows():
            r_skills = r_row["skills_norm"] or []
            ratio = skill_overlap_ratio(j_skills, r_skills)
            if ratio >= min_skill_overlap:
                overlaps.append((r_idx, ratio))

        if not overlaps:
            continue

        # Sort by overlap descending as a first proxy
        overlaps.sort(key=lambda x: x[1], reverse=True)
        cand_indices = [idx for idx, _ in overlaps]

        # If SBERT available, re-rank by cosine similarity
        if job_embs is not None and resume_embs is not None:
            j_vec = job_embs[j_idx]
            r_vecs = resume_embs[cand_indices]
            sims = np.dot(r_vecs, j_vec)
            order = np.argsort(-sims)
            ranked = [(cand_indices[i], overlaps[i][1], sims[i]) for i in order]
        else:
            ranked = [(idx, ratio, None) for idx, ratio in overlaps]

        # Take top-N
        ranked = ranked[:top_n]

        for r_idx, ov, sim in ranked:
            pairs.append(
                {
                    "job_index": j_idx,
                    "resume_index": r_idx,
                    "skill_overlap": ov,
                    "semantic_sim": float(sim) if sim is not None else np.nan,
                }
            )

    pairs_df = pd.DataFrame(pairs)
    print("Candidate pairs shape:", pairs_df.shape)
    return pairs_df


candidate_pairs = build_candidate_pairs(
    jobs_df=jobs,
    resumes_df=resumes,
    job_embs=job_embs,
    resume_embs=resume_embs,
    top_n=200,
    min_skill_overlap=0.3,
    # Có thể giới hạn số lượng job (ví dụ 500) nếu cần rút gọn để phù hợp tài nguyên tính toán
    max_jobs=None,
)


## 5. Feature engineering for Learning to Rank

Ở bước này, ta xây dựng các đặc trưng chính như mô tả trong phần Experimental Setup:
- **Skill Matching**: tỷ lệ kỹ năng trùng khớp
- **Experience Alignment**: chênh lệch số năm kinh nghiệm
- **Location & Preference**: match tỉnh/thành
- **Semantic Features**: độ tương đồng job–resume (nếu có SBERT)

Để notebook gọn và chạy được trên nhiều môi trường, ta hiện thực subset các đặc trưng cốt lõi, các đặc trưng mở rộng có thể bổ sung sau (title similarity, education matching, v.v.).


In [ ]:
# 5.1 Build feature matrix for candidate pairs


def build_features(pairs_df: pd.DataFrame, jobs_df: pd.DataFrame, resumes_df: pd.DataFrame) -> pd.DataFrame:
    feat_rows = []

    for _, row in pairs_df.iterrows():
        j = jobs_df.loc[row["job_index"]]
        r = resumes_df.loc[row["resume_index"]]

        job_exp = j.get("experience_years", np.nan)
        res_exp = r.get("Years of Experience", np.nan)
        try:
            res_exp = float(res_exp)
        except Exception:
            res_exp = np.nan

        exp_gap = np.nan
        if not pd.isna(job_exp) and not pd.isna(res_exp):
            exp_gap = abs(job_exp - res_exp)

        location_exact = 1.0 if j.get("location_norm", "") == r.get("city_norm", "") and j.get("location_norm", "") != "" else 0.0

        semantic = row.get("semantic_sim", np.nan)

        feat_rows.append(
            {
                "job_index": row["job_index"],
                "resume_index": row["resume_index"],
                "skill_overlap": row["skill_overlap"],
                "exp_gap": exp_gap if not pd.isna(exp_gap) else 0.0,
                "location_exact": location_exact,
                "semantic_sim": 0.0 if pd.isna(semantic) else float(semantic),
            }
        )

    feat_df = pd.DataFrame(feat_rows)

    # Min–max normalization for numeric features
    for col in ["skill_overlap", "exp_gap", "semantic_sim"]:
        col_min = feat_df[col].min()
        col_max = feat_df[col].max()
        if col_max > col_min:
            feat_df[col] = (feat_df[col] - col_min) / (col_max - col_min)
        else:
            feat_df[col] = 0.0

    print("Feature matrix shape:", feat_df.shape)
    return feat_df


features_df = build_features(candidate_pairs, jobs, resumes)
features_df.head()


## 6. Relevance label construction

Áp dụng schema gán nhãn 0–3 như mô tả trong Experimental Setup:
- **3 – Rất phù hợp**: `skill_similarity ≥ 0.8`, `experience_gap ≤ 1 năm`, `title_similarity ≥ 0.8`
- **2 – Phù hợp**: `skill_similarity ≥ 0.6`, `experience_gap ≤ 3 năm`
- **1 – Ít phù hợp**: `skill_similarity ≥ 0.4` **hoặc** `title_similarity ≥ 0.5`
- **0 – Không phù hợp**: còn lại.

Trong notebook này, ta dùng `skill_overlap` (sau chuẩn hóa) và `exp_gap` (chuẩn hóa ngược lại) làm xấp xỉ, và giữ slot cho `title_similarity` để có thể mở rộng sau nếu cần.


In [ ]:
# 6.1 Labeling function


def assign_relevance_labels(feat_df: pd.DataFrame) -> pd.DataFrame:
    df = feat_df.copy()

    # Approximate title similarity placeholder (0.0) – có thể cập nhật sau
    df["title_similarity"] = 0.0

    # Chuyển exp_gap đã chuẩn hóa về dạng "fit" (1 - gap_norm)
    exp_fit = 1.0 - df["exp_gap"]

    labels = []
    for _, row in df.iterrows():
        skill_sim = row["skill_overlap"]
        title_sim = row["title_similarity"]
        exp_score = exp_fit.loc[row.name]

        if skill_sim >= 0.8 and exp_score >= 0.8 and title_sim >= 0.8:
            lbl = 3
        elif skill_sim >= 0.6 and exp_score >= 0.6:
            lbl = 2
        elif skill_sim >= 0.4 or title_sim >= 0.5:
            lbl = 1
        else:
            lbl = 0
        labels.append(lbl)

    df["label"] = labels
    print("Label distribution:\n", df["label"].value_counts(normalize=True).sort_index())
    return df


labeled_df = assign_relevance_labels(features_df)
labeled_df.head()


## 7. Query-based formatting for Learning to Rank

Mỗi job được coi là một **query** (`qid`), mỗi resume là một **document** với nhãn relevance và vector đặc trưng:
- `qid = job_index`
- `label ∈ {0,1,2,3}`
- `features = [skill_overlap, exp_gap, location_exact, semantic_sim, title_similarity]`

Dữ liệu sau đó được tách train/validation/test theo **query** và chuyển thành format phù hợp cho LightGBM LambdaMART.


In [ ]:
# 7.1 Build LTR dataset with groups

ltr_df = labeled_df.copy()

# Define qid as job_index
ltr_df["qid"] = ltr_df["job_index"].astype(int)

feature_cols = ["skill_overlap", "exp_gap", "location_exact", "semantic_sim", "title_similarity"]

X = ltr_df[feature_cols].values
y = ltr_df["label"].values
qids = ltr_df["qid"].values

# Group sizes for LightGBM
unique_qids, group_sizes = np.unique(qids, return_counts=True)
print("#queries:", len(unique_qids), " total pairs:", len(ltr_df))

# Train/val/test split at query level
train_q, temp_q = train_test_split(unique_qids, test_size=0.30, random_state=RANDOM_SEED)
val_q, test_q = train_test_split(temp_q, test_size=0.50, random_state=RANDOM_SEED)

print("Train queries:", len(train_q), "Val queries:", len(val_q), "Test queries:", len(test_q))


def subset_by_qids(X, y, qids, target_qids):
    mask = np.isin(qids, target_qids)
    return X[mask], y[mask], qids[mask]


X_train, y_train, q_train = subset_by_qids(X, y, qids, train_q)
X_val, y_val, q_val = subset_by_qids(X, y, qids, val_q)
X_test, y_test, q_test = subset_by_qids(X, y, qids, test_q)


def group_from_qids(qids_array):
    _, counts = np.unique(qids_array, return_counts=True)
    return counts.tolist()


group_train = group_from_qids(q_train)
group_val = group_from_qids(q_val)
group_test = group_from_qids(q_test)

print("Train size:", X_train.shape, " Val size:", X_val.shape, " Test size:", X_test.shape)


## 8. Training Learning to Rank models (LambdaMART)

Ở bước này, ta huấn luyện mô hình **LambdaMART** với LightGBM như trong Experimental Setup:
- `num_leaves = 63`, `max_depth = -1`, `learning_rate = 0.05`, `n_estimators = 300`, `min_child_samples = 50`
- Sử dụng objective `lambdarank`, metric `ndcg`, group theo `qid`
- Early stopping theo NDCG trên tập validation.


In [ ]:
# 8.1 Train LambdaMART with LightGBM

train_dataset = lgb.Dataset(X_train, label=y_train, group=group_train, feature_name=feature_cols)
val_dataset = lgb.Dataset(X_val, label=y_val, group=group_val, feature_name=feature_cols, reference=train_dataset)

params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "ndcg_at": [10],
    "num_leaves": 63,
    "max_depth": -1,
    "learning_rate": 0.05,
    "n_estimators": 300,
    "min_child_samples": 50,
    "verbose": -1,
    "seed": RANDOM_SEED,
}

lgbm_model = lgb.train(
    params,
    train_dataset,
    valid_sets=[train_dataset, val_dataset],
    valid_names=["train", "val"],
    num_boost_round=300,
    early_stopping_rounds=20,
    verbose_eval=50,
)

print("Best iteration:", lgbm_model.best_iteration)


## 9. Evaluation: NDCG@10, MAP@10, Precision@10 và baseline rule-based

Ta đánh giá mô hình trên tập test ở mức query:
- Tính NDCG@10, MAP@10, Precision@10
- So sánh với baseline rule-based (dựa trên `skill_overlap`, `exp_gap`, `location_exact`)
- Thực hiện Wilcoxon signed-rank test để kiểm định khác biệt ý nghĩa thống kê (p < 0.05).


In [ ]:
# 9.1 Helper metrics and evaluation

from collections import defaultdict


def precision_at_k(y_true, y_score, k=10):
    order = np.argsort(-y_score)
    topk = y_true[order][:k]
    return np.mean(topk > 0) if len(topk) > 0 else 0.0


def average_precision_at_k(y_true, y_score, k=10):
    order = np.argsort(-y_score)
    y_sorted = y_true[order][:k]
    if not np.any(y_sorted > 0):
        return 0.0
    precisions = []
    rel_count = 0
    for i, rel in enumerate(y_sorted, start=1):
        if rel > 0:
            rel_count += 1
            precisions.append(rel_count / i)
    return float(np.mean(precisions)) if precisions else 0.0


def evaluate_per_query(y_true, y_score, qids, k=10):
    ndcgs, maps, precs = [], [], []
    per_query_scores = {}

    for q in np.unique(qids):
        mask = qids == q
        y_q = y_true[mask]
        s_q = y_score[mask]

        if len(y_q) == 0:
            continue

        # NDCG@k
        ndcg = ndcg_score(y_true=[y_q], y_score=[s_q], k=k)
        m_ap = average_precision_at_k(y_q, s_q, k=k)
        p_at = precision_at_k(y_q, s_q, k=k)

        ndcgs.append(ndcg)
        maps.append(m_ap)
        precs.append(p_at)

        per_query_scores[q] = {"ndcg": ndcg, "map": m_ap, "p@k": p_at}

    return {
        "ndcg@k": float(np.mean(ndcgs)),
        "map@k": float(np.mean(maps)),
        "p@k": float(np.mean(precs)),
        "per_query": per_query_scores,
    }


# Predict with LambdaMART
pred_test_lgbm = lgbm_model.predict(X_test, num_iteration=lgbm_model.best_iteration)
metrics_lgbm = evaluate_per_query(y_test, pred_test_lgbm, q_test, k=10)
print("LambdaMART test metrics:", metrics_lgbm["ndcg@k"], metrics_lgbm["map@k"], metrics_lgbm["p@k"])


# Simple rule-based baseline score
rule_score_test = (
    0.6 * X_test[:, feature_cols.index("skill_overlap")] +
    0.2 * (1.0 - X_test[:, feature_cols.index("exp_gap")]) +
    0.2 * X_test[:, feature_cols.index("location_exact")]
)

metrics_rule = evaluate_per_query(y_test, rule_score_test, q_test, k=10)
print("Rule-based test metrics:", metrics_rule["ndcg@k"], metrics_rule["map@k"], metrics_rule["p@k"])


# Wilcoxon signed-rank test on per-query NDCG
common_qids = sorted(set(metrics_lgbm["per_query"].keys()) & set(metrics_rule["per_query"].keys()))
if common_qids:
    ndcg_lgbm = [metrics_lgbm["per_query"][q]["ndcg"] for q in common_qids]
    ndcg_rule = [metrics_rule["per_query"][q]["ndcg"] for q in common_qids]
    stat, p_val = wilcoxon(ndcg_lgbm, ndcg_rule)
    print("Wilcoxon signed-rank test on NDCG@10: stat=", stat, " p=", p_val)
else:
    print("No common queries for Wilcoxon test.")
